## PUNTO 2:

Para el punto segun el siguiente enunciado:
>Suponga que desea utilizar Programación Genética para encontrar el diseño de un circuito lógico, tome como, ejemplo el codificador de 7 segmentos. Describa el conjunto de terminales, el conjunto de funciones y la función de aptitud. Use una librería de Python.

Para el decodificador de 7 segmentos sae utilizará la teoría que se encontró en la siguiente página [link](https://wilaebaelectronica.blogspot.com/2017/01/decodificador-bcd-a-7-segmentos.html), de donde se tomó la siguiente tabla de verdad que se usará para evaluar el fitness de las soluciones generadas por el algoritmo:


![Texto alternativo](figura.png)

A diferencia de la imagen mostrada se usa la siguiente convención para los números del 10 al 15:

| Número Decimal | Binario (A,B,C,D) | Segmentos Encendidos (S_0 a S_6) | Representación |
|----------------|--------------------|----------------------------------|----------------|
| 10             | 1010              | [1, 1, 1, 0, 1, 1, 1]          | A              |
| 11             | 1011              | [0, 0, 1, 1, 0, 1, 1]          | b              |
| 12             | 1100              | [1, 0, 0, 1, 1, 1, 0]          | C              |
| 13             | 1101              | [0, 1, 1, 1, 1, 0, 1]          | d              |
| 14             | 1110              | [1, 0, 0, 1, 1, 1, 1]          | E              |
| 15             | 1111              | [1, 0, 0, 0, 1, 1, 1]          | F              |

Para el ejercicio se utilizarán:
* **CONJUNTO DE TERMINALES**: Basado en la imagen las entradas serán ***A,B,C,D*** que serán entradas binarias para representar el numero que se desea representar
* **CONJUNTO DE FUNCIONES**: Por simplicidad en esta función se utilizarán unicamente las funciones básicas las cuales serán ***AND, OR*** y ***NOT***
* **FUNCIÓN DE APTITUD**: Estará dada por la tasa de aceirtos que se tuvó el circuito diseñado respecto a la sálida esperada: $\frac{\#aciertos}{16}$ dónde $16=2^4$

Para la solución de este ejercicio se utilizó la biblioteca [DEAP](https://deap.readthedocs.io/en/master/) de python

In [7]:
pip install deap

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
from deap import base, creator, tools, gp, algorithms
import operator
import random
import numpy as np
# Tabla de verdad para el codificador de 7 segmentos
truth_table = {
    (0, 0, 0, 0): [1, 1, 1, 1, 1, 1, 0],
    (0, 0, 0, 1): [0, 1, 1, 0, 0, 0, 0],
    (0, 0, 1, 0): [1, 1, 0, 1, 1, 0, 1],
    (0, 0, 1, 1): [1, 1, 1, 1, 0, 0, 1],
    (0, 1, 0, 0): [0, 1, 1, 0, 0, 1, 1],
    (0, 1, 0, 1): [1, 0, 1, 1, 0, 1, 1],
    (0, 1, 1, 0): [1, 0, 1, 1, 1, 1, 1],
    (0, 1, 1, 1): [1, 1, 1, 0, 0, 0, 0],
    (1, 0, 0, 0): [1, 1, 1, 1, 1, 1, 1],
    (1, 0, 0, 1): [1, 1, 1, 1, 0, 1, 1],
    (1, 0, 1, 0): [1, 1, 1, 0, 1, 1, 1],
    (1, 0, 1, 1): [0, 0, 1, 1, 0, 1, 1],
    (1, 1, 0, 0): [1, 1, 1, 0, 0, 1, 1],
    (1, 1, 0, 1): [1, 1, 1, 1, 1, 0, 0],
    (1, 1, 1, 0): [1, 1, 1, 1, 1, 1, 1],
    (1, 1, 1, 1): [1, 1, 1, 1, 0, 0, 1]
}


# Definir el conjunto de terminales y funciones
pset = gp.PrimitiveSet("MAIN", 4)  # 4 Entradas: A, B, C, D
pset.addPrimitive(operator.and_, 2)  # AND con 2 entradas
pset.addPrimitive(operator.or_, 2)   # OR con 2 entradas
pset.addPrimitive(operator.not_, 1)  # NOT con 1 entrada
pset.addEphemeralConstant("rand", lambda: random.randint(0, 1))  # Constantes aleatorias
pset.renameArguments(ARG0="A", ARG1="B", ARG2="C", ARG3="D")

# Crear la clase de aptitud y el individuo
creator.create("FitnessMax", base.Fitness, weights=(1.0,))  # Maximizar aptitud
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

# Configurar la caja de herramientas
toolbox = base.Toolbox()
toolbox.register("expr", gp.genFull, pset=pset, min_=1, max_=3)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

# Función de aptitud
def eval_circuit(individual):
    func = toolbox.compile(expr=individual)
    correct_outputs = 0
    
    for inputs, expected_output in truth_table.items():
        output = [func(*inputs) for _ in range(7)]
        if output == expected_output:
            correct_outputs += 1
            
    return correct_outputs / len(truth_table),

toolbox.register("evaluate", eval_circuit)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr, pset=pset)

# Configurar algoritmo genético
toolbox.register("map", map)

# Configurar el algoritmo evolutivo
def main():
    random.seed(42)
    population = toolbox.population(n=300)
    hof = tools.HallOfFame(1)
    
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
    
    algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=40, 
                        stats=stats, halloffame=hof, verbose=True)
    
    return population, stats, hof

# Ejecutar y visualizar el mejor individuo
if __name__ == "__main__":
    population, stats, hof = main()
    print("Best individual:", hof[0])


c:\Users\david\AppData\Local\Programs\Python\Python311\Lib\site-packages\deap\gp.py:254: RuntimeWarning: Ephemeral rand function cannot be pickled because its generating function is a lambda function. Use functools.partial instead.
  warnings.warn("Ephemeral {name} function cannot be "
c:\Users\david\AppData\Local\Programs\Python\Python311\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\david\AppData\Local\Programs\Python\Python311\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	avg      	std     	min	max  
0  	300   	0.0608333	0.049711	0  	0.125
1  	194   	0.0891667	0.0446903	0  	0.125
2  	188   	0.103125 	0.0402224	0  	0.125
3  	169   	0.108333 	0.0369168	0  	0.125
4  	187   	0.11     	0.0343693	0  	0.125
5  	188   	0.1125   	0.0330719	0  	0.125
6  	185   	0.114375 	0.0289418	0  	0.125
7  	173   	0.1175   	0.0249165	0  	0.125
8  	183   	0.112708 	0.0325554	0  	0.125
9  	155   	0.116875 	0.0254977	0  	0.125
10 	182   	0.1175   	0.0254337	0  	0.125
11 	164   	0.120833 	0.0171796	0  	0.125
12 	182   	0.11875  	0.0231053	0  	0.125
13 	189   	0.120833 	0.0171796	0  	0.125
14 	193   	0.117292 	0.0266235	0  	0.125
15 	177   	0.117917 	0.0228636	0  	0.125
16 	152   	0.119792 	0.0200639	0  	0.125
17 	166   	0.120208 	0.0214199	0  	0.125
18 	174   	0.119792 	0.0200639	0  	0.125
19 	176   	0.12     	0.0216747	0  	0.125
20 	186   	0.120833 	0.01932  	0  	0.125
21 	192   	0.11875  	0.0247382	0  	0.125
22 	189   	0.122083 	0.0150289	0  	0.125
23 	181   	0.12229

## PUNTO 3

De acuero al enunciado:

>Suponga que tiene un robot que le entrega galletas al grupo de ingenieros de diseño de robots. Programe por PG el recorrido del robot, teniendo en cuenta que cada vez que un ingeniero recibe una galleta gana puntos. Los ingenieros están distribuidos en una sala cuadrada. Defina, conjunto de terminales, conjunto de funciones y función de aptitud.
x
Para el ejercicio se utilizarán:
* **CONJUNTO DE TERMINALES**: 
    * "UP": Mover una celda hacia arriba.
    * "DOWN": Mover una celda hacia abajo.
    * "LEFT": Mover una celda hacia la izquierda.
    * "RIGHT": Mover una celda hacia la derecha.
* **CONJUNTO DE FUNCIONES**:
    * if_else(cond, action1, action2): Evalúa una condición cond y ejecuta action1 si es verdadera, o action2 si es falsa.
    * sequence(action1, action2): Ejecuta action1 seguido de action2.
* **FUNCIÓN DE APTITUD**: 
    * Puntos ganados: La cantidad de ingenieros visitados por el robot.
    * Eficiencia del recorrido: Penaliza trayectorias que sean demasiado largas o que salgan de los límites del mapa.

    Se podría resumir como:
    1. Inicializa los puntos en 0.
    2. Ejecuta el recorrido generado por el árbol de PG.
    3. Cada vez que el robot visita un ingeniero, suma puntos y marca al ingeniero como visitado.
    4. Penaliza movimientos fuera de los límites del mapa.

Para minimizar las acciones se limitarán el numero de pasos que puede dar el robot a 20

In [33]:
import random
import numpy as np

# Tamaño del mapa
MAP_SIZE = 5
engineers = [(0, 3), (1, 1), (2, 2), (3, 4), (4, 0)]  # Posiciones de los ingenieros

# Definir las acciones posibles para el robot sin 'STAY'
MOVES = ['UP', 'DOWN', 'LEFT', 'RIGHT']

# Función para generar una secuencia aleatoria de movimientos
def generate_random_sequence(length=20):  # Cambiado a 20 movimientos
    return [random.choice(MOVES) for _ in range(length)]

# Función para mover el robot en el mapa
def move_robot(position, action):
    if action == 'UP' and position[0] > 0:
        position[0] -= 1
    elif action == 'DOWN' and position[0] < MAP_SIZE - 1:
        position[0] += 1
    elif action == 'LEFT' and position[1] > 0:
        position[1] -= 1
    elif action == 'RIGHT' and position[1] < MAP_SIZE - 1:
        position[1] += 1
    return position

# Función para imprimir el mapa con la posición actual del robot y los ingenieros
def print_map(robot_position, path):
    print("\nMapa del recorrido:")
    for i in range(MAP_SIZE):
        row = ""
        for j in range(MAP_SIZE):
            if (i, j) == tuple(robot_position):
                row += " R "  # Representa el robot
            elif (i, j) in engineers:
                row += " E "  # Representa un ingeniero
            elif (i, j) in path:
                row += " * "  # Representa el camino recorrido por el robot
            else:
                row += " . "  # Representa un espacio vacío
        print(row)
    print()

# Función para evaluar la aptitud de una secuencia de movimientos
def evaluate_sequence(sequence):
    position = [0, 0]  # Posición inicial del robot
    visited = set()  # Ingenieros visitados
    path = []  # Camino recorrido
    points = 0  # Puntos obtenidos

    # Ejecutar la secuencia de movimientos
    for action in sequence:
        position = move_robot(position, action)
        path.append(tuple(position))  # Agregar la nueva posición al camino recorrido
        
        # Verificar si el robot visita un ingeniero
        if tuple(position) in engineers and tuple(position) not in visited:
            visited.add(tuple(position))
            points += 1

    return points, path

# Algoritmo Genético
def genetic_algorithm(population_size=100, generations=50, mutation_rate=0.1, crossover_rate=0.7):
    # Inicializar población con secuencias aleatorias
    population = [generate_random_sequence() for _ in range(population_size)]
    
    best_fitness = 0
    best_solution = None
    best_path = []

    for generation in range(generations):
        # Evaluar la aptitud de la población
        fitness = [evaluate_sequence(individual) for individual in population]
        points = [fitness_ind[0] for fitness_ind in fitness]
        paths = [fitness_ind[1] for fitness_ind in fitness]
        
        # Selección: elegir los mejores individuos para reproducir
        selected_individuals = select_population(population, points, population_size)
        
        # Crossover: generar la próxima generación a partir de los mejores
        offspring = crossover_population(selected_individuals, crossover_rate)
        
        # Mutación: introducir cambios aleatorios
        population = mutate_population(offspring, mutation_rate)
        
        # Mejor puntuación en esta generación
        max_points = max(points)
        if max_points > best_fitness:
            best_fitness = max_points
            best_solution = population[np.argmax(points)]
            best_path = paths[np.argmax(points)]

        # Imprimir el mejor individuo de la generación
        #print(f"Generación {generation + 1}: Mejor puntuación {max_points}")
        
    # Imprimir el mapa del recorrido paso a paso para la mejor solución
    print("Mejor solución encontrada:", best_solution)
    print("Mejor puntuación:", best_fitness)
    print("Recorrido del robot paso a paso:")
    position = [0, 0]  # Posición inicial del robot
    path = []
    for action in best_solution:
        position = move_robot(position, action)
        path.append(tuple(position))
        print_map(position, path)  # Imprimir mapa en cada paso

    return best_solution, best_fitness

# Función de selección de la población
def select_population(population, fitness, population_size):
    # Selección por torneo: elegir dos individuos al azar y elige al mejor
    selected = []
    for _ in range(population_size // 2):
        tournament = random.sample(list(zip(population, fitness)), 2)
        tournament.sort(key=lambda x: x[1], reverse=True)
        selected.append(tournament[0][0])
    return selected

# Función de crossover (cruce) entre dos individuos
def crossover_population(population, crossover_rate):
    offspring = []
    for i in range(0, len(population), 2):
        if random.random() < crossover_rate:
            parent1 = population[i]
            parent2 = population[i + 1]
            crossover_point = random.randint(1, len(parent1) - 1)
            child1 = parent1[:crossover_point] + parent2[crossover_point:]
            child2 = parent2[:crossover_point] + parent1[crossover_point:]
            offspring.extend([child1, child2])
        else:
            offspring.extend([population[i], population[i + 1]])
    return offspring

# Función de mutación: realiza cambios aleatorios en la secuencia de movimientos
def mutate_population(population, mutation_rate):
    mutated_population = []
    for individual in population:
        if random.random() < mutation_rate:
            mutation_point = random.randint(0, len(individual) - 1)
            individual[mutation_point] = random.choice(MOVES)
        mutated_population.append(individual)
    return mutated_population

# Ejecutar el algoritmo genético
best_solution, best_fitness =genetic_algorithm(population_size=100, generations=100, mutation_rate=0.1, crossover_rate=0.7)


Mejor solución encontrada: ['UP', 'RIGHT', 'DOWN', 'DOWN', 'RIGHT', 'UP', 'UP', 'UP', 'RIGHT', 'DOWN', 'RIGHT', 'UP', 'RIGHT', 'LEFT', 'UP', 'RIGHT', 'LEFT', 'DOWN', 'RIGHT', 'RIGHT']
Mejor puntuación: 4
Recorrido del robot paso a paso:

Mapa del recorrido:
 R  .  .  E  . 
 .  E  .  .  . 
 .  .  E  .  . 
 .  .  .  .  E 
 E  .  .  .  . 


Mapa del recorrido:
 *  R  .  E  . 
 .  E  .  .  . 
 .  .  E  .  . 
 .  .  .  .  E 
 E  .  .  .  . 


Mapa del recorrido:
 *  *  .  E  . 
 .  R  .  .  . 
 .  .  E  .  . 
 .  .  .  .  E 
 E  .  .  .  . 


Mapa del recorrido:
 *  *  .  E  . 
 .  E  .  .  . 
 .  R  E  .  . 
 .  .  .  .  E 
 E  .  .  .  . 


Mapa del recorrido:
 *  *  .  E  . 
 .  E  .  .  . 
 .  *  R  .  . 
 .  .  .  .  E 
 E  .  .  .  . 


Mapa del recorrido:
 *  *  .  E  . 
 .  E  R  .  . 
 .  *  E  .  . 
 .  .  .  .  E 
 E  .  .  .  . 


Mapa del recorrido:
 *  *  R  E  . 
 .  E  *  .  . 
 .  *  E  .  . 
 .  .  .  .  E 
 E  .  .  .  . 


Mapa del recorrido:
 *  *  R  E  . 
 .  E  *  . 